# Training Prediction Models Directly Within PostgreSQL Using EvaDB
In this tutorial, we'll harness EvaDB's model training capabilities to predict home rental prices, showcasing how EvaDB seamlessly integrates AI into your PostgreSQL database.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/staging/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"/> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/staging/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"/> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/staging/tutorials/17-home-rental-prediction.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

## Setup

### Install and Launch the PostgreSQL Server

To kick things off, we'll start by setting up the PostgreSQL database backend. If you already have a PostgreSQL server up and running, you can skip this step and proceed directly to [installing EvaDB](#install-evadb).

In [1]:
!apt -qq install postgresql
!service postgresql start

The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 11 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package logrotate.
(Reading database ... 120880 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1.1

### Create User and Database

In [2]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


### Prettify  Output

In [3]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
def pretty_print(df):
    return display(HTML( df.to_html().replace("\\n","<br>")))

### Installing EvaDB
<a id='install_evadb'></a>
We install EvaDB along with the necessary PostgreSQL and Ludwig dependencies.

In [5]:
%pip install --quiet "evadb[postgres,xgboost,ludwig]"

import evadb
cursor = evadb.connect().cursor()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.7/578.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 757kB/s] 
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


## Load data into PostgresSQL


### Setting up a Data Source in EvaDB
To establish a direct connection between EvaDB and underlying database systems such as PostgreSQL, we will create a data source. This process entails supplying EvaDB with the connection credentials for the active PostgreSQL server.

In [6]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


### Loading Home Property Sales Data from CSV into PostgreSQL

In this step, we will import the [House Property Sales](https://www.kaggle.com/datasets/htagholdings/property-sales?resource=download) dataset into our PostgreSQL database. If you already have the data stored in PostgreSQL and are ready to proceed with the prediction model training, feel free to skip this section and head directly to the [model training process](#train-the-prediction-model).


In [7]:
!mkdir -p content
!wget -nc -O /content/home_rentals.csv https://www.dropbox.com/scl/fi/gy2682i66a8l2tqsowm5x/home_rentals.csv?rlkey=e080k02rv5205h4ullfjdr8lw&raw=1

--2023-11-28 19:28:27--  https://www.dropbox.com/scl/fi/gy2682i66a8l2tqsowm5x/home_rentals.csv?rlkey=e080k02rv5205h4ullfjdr8lw
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdac3aa9e9d3c29ba317cdba528.dl.dropboxusercontent.com/cd/0/inline/CIZBjRk_219UjBC3q8uyjzPSHGVgfxu9ZtCf_WESgWXjOAd_foWNWFvmDso2nw60coSSOeaZ3tOqD8nTdEDbRY6ohlvIm9yJIi9K2DrWPIx8uQvMqiaO2v_76rb3IEERLR399gA2D8NkCgA7aBIpNU62/file# [following]
--2023-11-28 19:28:28--  https://ucdac3aa9e9d3c29ba317cdba528.dl.dropboxusercontent.com/cd/0/inline/CIZBjRk_219UjBC3q8uyjzPSHGVgfxu9ZtCf_WESgWXjOAd_foWNWFvmDso2nw60coSSOeaZ3tOqD8nTdEDbRY6ohlvIm9yJIi9K2DrWPIx8uQvMqiaO2v_76rb3IEERLR399gA2D8NkCgA7aBIpNU62/file
Resolving ucdac3aa9e9d3c29ba317cdba528.dl.dropboxusercontent.com (ucdac3aa9e9d3c29ba317cdba528.dl.dropboxusercontent.com)... 162.125.5.15, 26

In [8]:
cursor.query("""
  USE postgres_data {
    CREATE TABLE IF NOT EXISTS home_rentals (
      number_of_rooms INT,
      number_of_bathrooms INT,
      sqft INT,
      location VARCHAR(128),
      days_on_market INT,
      initial_price INT,
      neighborhood VARCHAR(128),
      rental_price FLOAT
    )
  }
""").df()

,status
0,success


In [9]:
cursor.query("""
  USE postgres_data {
    COPY home_rentals(number_of_rooms, number_of_bathrooms, sqft, location, days_on_market, initial_price, neighborhood, rental_price)
    FROM '/content/home_rentals.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

,status
0,success


### Preview the Data


Within the home_rentals table, there are 8 columns at our disposal. Our objective is to utilize the remaining 7 columns to make predictions for the rental_price.

In [10]:
cursor.query("SELECT * FROM postgres_data.home_rentals LIMIT 3;").df()

,rental_price,number_of_bathrooms,sqft,initial_price,number_of_rooms,days_on_market,location,neighborhood
0,2167.0,1,674,2167,1,1,good,downtown
1,1883.0,1,554,1883,1,19,poor,westbrae
2,2431.0,1,529,2431,0,3,great,south_side


## Training Model

Next, we employ EvaDB to facilitate the training of an ML model, which will enable us to predict `home rental prices`.

### Train the prediction Model using Ludwig
For this purpose, we harness the capabilities of the [ludwig](https://ludwig.ai/latest/) engine to train our prediction model. We employ the `automl` feature to automatically determine the optimal hyperparameters. Keep in mind that `TIME_LIMIT` specifies the time budget allocated for the training process.

In [ ]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE Ludwig
  PREDICT 'rental_price'
  TIME_LIMIT 3600;
""").df()

### Train the prediction Model using XGBoost
With XGBoost integration, we harness the capabilities of the [XGBoost](https://xgboost.readthedocs.io/en/stable/) engine to train our prediction model. We employ the `Flaml` AutoML model to automatically determine the optimal hyperparameters. We can utilize `TIME_LIMIT` to specify the time budget allocated for the training process. Similarly you can use the `METRIC` parameter to determine the accuracy/error metric for training the model. `TASK` parameter can be used to specify whether you want to perform a `classification` task or `regression` task.

In [ ]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE XGBoost
  PREDICT 'rental_price'
  TIME_LIMIT 180
  METRIC 'rmse'
  TASK 'regression';
""").df()

[flaml.automl.logger: 10-26 18:42:28] {1679} INFO - task = regression
[flaml.automl.logger: 10-26 18:42:28] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 10-26 18:42:28] {1788} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 10-26 18:42:28] {1900} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl.logger: 10-26 18:42:28] {2218} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 10-26 18:42:28] {2344} INFO - Estimated sufficient time budget=1240s. Estimated necessary time budget=1s.
[flaml.automl.logger: 10-26 18:42:28] {2391} INFO -  at 0.2s,	estimator xgboost's best error=873.7432,	best estimator xgboost's best error=873.7432
[flaml.automl.logger: 10-26 18:42:28] {2218} INFO - iteration 1, current learner xgboost
[flaml.automl.logger: 10-26 18:42:28] {2391} INFO -  at 0.3s,	estimator xgboost's best error=873.7432,	best estimator xgboost's best error=873.7432
[flaml.automl.logger: 10-26 18:42:28] {2218} INFO - iteration 2, current 

,0
0,Function PredictHouseRent added to the database.


### Train the prediction Model using Sklearn
With Sklearn Flaml integration, we harness the capabilities of the [Sklearn](https://scikit-learn.org/stable/) engine to train our prediction model. We employ the `Flaml` AutoML model to automatically determine the optimal hyperparameters. We can utilize `TIME_LIMIT` to specify the time budget allocated for the training process. Similarly you can use the `METRIC` parameter to determine the accuracy/error metric for training the model. `TASK` parameter can be used to specify whether you want to perform a `classification` task or `regression` task. Additionally you can use the `MODEL` parameter to specify the Sklearn ML model that you would like to use for training. You can set the `MODEL` parameter to with `rf`, `extra_tree` or `kneighbor`.

In [11]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE Sklearn
  PREDICT 'rental_price'
  TIME_LIMIT 180
  METRIC 'rmse'
  TASK 'regression'
  MODEL 'rf';
""").df()

[flaml.automl.logger: 11-28 19:41:22] {1679} INFO - task = regression
[flaml.automl.logger: 11-28 19:41:22] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 11-28 19:41:22] {1788} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 11-28 19:41:22] {1900} INFO - List of ML learners in AutoML Run: ['rf']
[flaml.automl.logger: 11-28 19:41:22] {2218} INFO - iteration 0, current learner rf
[flaml.automl.logger: 11-28 19:41:23] {2344} INFO - Estimated sufficient time budget=3889s. Estimated necessary time budget=4s.
[flaml.automl.logger: 11-28 19:41:23] {2391} INFO -  at 0.5s,	estimator rf's best error=291.9744,	best estimator rf's best error=291.9744
[flaml.automl.logger: 11-28 19:41:23] {2218} INFO - iteration 1, current learner rf
[flaml.automl.logger: 11-28 19:41:23] {2391} INFO -  at 0.9s,	estimator rf's best error=99.8456,	best estimator rf's best error=99.8456
[flaml.automl.logger: 11-28 19:41:23] {2218} INFO - iteration 2, current learner rf
[flaml.automl.logger: 11-2

,0
0,Function PredictHouseRent added to the database.
1,Validation Score: 5.499365480177278
2,Training time: 171 secs.


In [17]:
cursor.query("""
  CREATE OR REPLACE FUNCTION PredictHouseRent FROM
  ( SELECT * FROM postgres_data.home_rentals )
  TYPE Sklearn
  PREDICT 'rental_price'
  TIME_LIMIT 30
  METRIC 'r2'
  TASK 'regression'
  MODEL 'extra_tree';
""").df()

[flaml.automl.logger: 11-28 19:52:18] {1679} INFO - task = regression
[flaml.automl.logger: 11-28 19:52:18] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 11-28 19:52:18] {1788} INFO - Minimizing error metric: 1-r2
[flaml.automl.logger: 11-28 19:52:18] {1900} INFO - List of ML learners in AutoML Run: ['extra_tree']
[flaml.automl.logger: 11-28 19:52:18] {2218} INFO - iteration 0, current learner extra_tree
[flaml.automl.logger: 11-28 19:52:18] {2344} INFO - Estimated sufficient time budget=2913s. Estimated necessary time budget=3s.
[flaml.automl.logger: 11-28 19:52:18] {2391} INFO -  at 0.3s,	estimator extra_tree's best error=0.0717,	best estimator extra_tree's best error=0.0717
[flaml.automl.logger: 11-28 19:52:18] {2218} INFO - iteration 1, current learner extra_tree
[flaml.automl.logger: 11-28 19:52:18] {2391} INFO -  at 0.6s,	estimator extra_tree's best error=0.0098,	best estimator extra_tree's best error=0.0098
[flaml.automl.logger: 11-28 19:52:18] {2218} INFO - iteratio

,0
0,Function PredictHouseRent added to the database.
1,Validation Score: 6.991127421329679e-06
2,Training time: 30 secs.


### Utilizing the Prediction Model
Following the model training, we proceed to employ the `PredictHouseRent`` model to make predictions for home rental prices.

In [18]:
cursor.query("SELECT PredictHouseRent(*) FROM postgres_data.home_rentals LIMIT 10;").df()

,rental_price
0,2167.179067
1,1881.860347
2,2430.833594
3,5510.144792
4,2272.339955
5,4126.598719
6,2224.262202
7,2104.066761
8,3860.835156
9,2040.849091


We have the option to utilize a `LATERAL JOIN` to compare the actual rental prices in the `home_rentals` dataset with the predicted rental prices generated by the trained model, `PredictHouseRent`.

In [19]:
cursor.query("""
  SELECT rental_price, predicted_rental_price FROM postgres_data.home_rentals
  JOIN LATERAL PredictHouseRent(*) AS Predicted(predicted_rental_price) LIMIT 10;
""").df()

,rental_price,predicted_rental_price
0,2167.000,2167.179067
1,1883.000,1883.452487
2,2431.000,2430.646094
3,5510.000,5509.533854
4,2272.000,2272.023586
5,4123.812,4123.744156
6,2224.000,2224.262202
7,2104.000,2103.969496
8,3861.000,3861.523438
9,2041.000,2040.849091
